In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

2025-01-03 10:25:55.476001: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-03 10:25:55.627054: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 10:25:56.311350: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-03 10:25:56.311396: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-03 10:25:56.313407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [11]:
def create_model():
    base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(96, 96, 3)
    )
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(43, activation='softmax')(x)

    return Model(inputs=base_model.input, outputs=predictions)


In [12]:
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2]
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255
    )

    return train_datagen, val_datagen

In [13]:
def train_model(model, train_generator, validation_generator, epochs=50):
    checkpoint = tf.keras.callbacks.ModelCheckpoint (
        "./checkpoints/traffic_sign_classification_resnet50_{epoch:02d}_{val_accuracy:.3f}.keras",
        save_best_only=True,
        monitor="val_accuracy"
    )

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks=[checkpoint]
    )

    return history

In [14]:
model = create_model()

train_datagen, val_datagen = create_data_generators()

In [16]:
train_generator = train_datagen.flow_from_directory(
    './app/data/train',  
    target_size=(96, 96),
    batch_size=32,
    subset='training',
    class_mode='categorical'
)

validation_generator = val_datagen_datagen.flow_from_directory(
    './app/data/train',  
    target_size=(96, 96),
    batch_size=32,
    subset='validation',
    class_mode='categorical'
)

FileNotFoundError: [Errno 2] No such file or directory: './app/data/train'

In [ ]:

history = train_model(model, train_generator=train_generator, validation_generator=validation_generator)